In [7]:
import sys
sys.stdout.flush()

# Or force flush after prints
print("Your message", flush=True)

import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
from pathlib import Path
from datetime import datetime
import math

import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import (
    Flatten, Dropout, Dense, Input, GlobalAveragePooling2D, Conv2D,
    BatchNormalization, Activation, MaxPooling2D, Lambda
)
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import (
    ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, LearningRateScheduler
)
from tensorflow.keras.applications import MobileNetV2, EfficientNetB0

from sklearn.metrics import confusion_matrix, classification_report, precision_recall_fscore_support
from sklearn.utils.class_weight import compute_class_weight

# Enable mixed precision
from tensorflow.keras import mixed_precision

print("✅ All libraries imported successfully")


Your message
✅ All libraries imported successfully


In [8]:
# Check all available devices

import socket


print("Available devices:")
print(tf.config.list_physical_devices())

# Check GPU specifically
gpus = tf.config.list_physical_devices('GPU')
print(f"GPUs found: {len(gpus)}")
for gpu in gpus:
    print(f"  {gpu}")
    
hostname = socket.gethostname()
print("Hostname:", hostname)

# Check if GPU is actually being used
print("Built with CUDA:", tf.test.is_built_with_cuda())
print("GPU available:", tf.test.is_gpu_available())
# Enable mixed precision only if a GPU is available
if tf.config.list_physical_devices('GPU'):
    mixed_precision.set_global_policy('mixed_float16')
    print("Mixed precision enabled: mixed_float16")
else:
    if hostname == "xscape7x":
        # Enable oneDNN optimizations
        tf.config.optimizer.set_jit(True)  # XLA compilation
        os.environ['TF_ENABLE_ONEDNN_OPTS'] = '1'

        # Set thread count for CPU
        tf.config.threading.set_inter_op_parallelism_threads(4)
        tf.config.threading.set_intra_op_parallelism_threads(4)

        # Use mixed precision for CPU
        from tensorflow.keras import mixed_precision
        policy = mixed_precision.Policy('mixed_float16')
        mixed_precision.set_global_policy(policy)
        mixed_precision.set_global_policy('float32')
    else:
        print("No GPU detected — using float32")
        print("No acceleration available — using float32")
        
print("Global policy:", mixed_precision.global_policy())




Available devices:
[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]
GPUs found: 0
Hostname: xscape7x
Built with CUDA: False
GPU available: False
Global policy: <DTypePolicy "float32">


In [9]:


# =========================================================================
# Clean Configuration System
# =========================================================================

class ModelConfig:
    """Clean configuration class with backbone-specific presets"""
    
    BACKBONE_PRESETS = {
        'custom_cnn': {
            'picture_size': 48,
            'color_mode': 'grayscale',
            'batch_size': 64,
            'epochs': 50,
            'learning_rate': 0.001,
            'dropout_rate': 0.25,
            'dense_units': [512],
            'aug_level': 'light',
            'use_class_weights': True,
            'use_lr_schedule': True,
        },
        'mobilenet': {
            'picture_size': 96,
            'color_mode': 'rgb',
            'batch_size': 32,
            'epochs': 30,
            'learning_rate': 0.0001,
            'dropout_rate': 0.3,
            'dense_units': [512, 256],
            'aug_level': 'strong',
            'use_class_weights': True,
            'use_lr_schedule': True,
            'fine_tune': True,
            'fine_tune_epochs': 20,
            'fine_tune_unfreeze_layers': 30,
        },
        'efficientnet': {
            'picture_size': 128,
            'color_mode': 'rgb',
            'batch_size': 16,
            'epochs': 40,
            'learning_rate': 0.0001,
            'dropout_rate': 0.4,
            'dense_units': [1024, 512],
            'aug_level': 'strong',
            'use_class_weights': True,
            'use_lr_schedule': True,
            'fine_tune': True,
            'fine_tune_epochs': 25,
            'fine_tune_unfreeze_layers': 50,
        }
    }
    
    AUGMENTATION_LEVELS = {
        'none': dict(rescale=1./255),
        'light': dict(
            rescale=1./255,
            width_shift_range=0.1,
            height_shift_range=0.1,
            zoom_range=0.2,
            horizontal_flip=True
        ),
        'strong': dict(
            rescale=1./255,
            rotation_range=20,
            width_shift_range=0.2,
            height_shift_range=0.2,
            shear_range=0.15,
            zoom_range=0.2,
            brightness_range=[0.7, 1.3],
            horizontal_flip=True,
            fill_mode='nearest'
        ),
    }
    
    def __init__(self, backbone='mobilenet'):
        """Initialize with backbone preset"""
        if backbone not in self.BACKBONE_PRESETS:
            raise ValueError(f"Backbone must be one of {list(self.BACKBONE_PRESETS.keys())}")
        
        self.backbone = backbone
        self.config = self.BACKBONE_PRESETS[backbone].copy()
        self.config['backbone'] = backbone
        self.config['no_of_classes'] = 7
        
    def get(self, key, default=None):
        """Get configuration value"""
        return self.config.get(key, default)
    
    def set(self, key, value):
        """Set configuration value"""
        self.config[key] = value
    
    def to_dict(self):
        """Return full configuration"""
        return self.config


In [10]:

# =========================================================================
# Data Loading Utilities
# =========================================================================

def get_data_path():
    """Get data path based on environment"""
    if os.path.exists('/kaggle/input'):
        import kagglehub
        image_path = kagglehub.dataset_download("jonathanoheix/face-expression-recognition-dataset")
        return os.path.join(image_path, "images")
    else:
        return "data/images/"

def create_data_generators(config, data_path):
    """Create clean data generators"""
    
    # Get augmentation parameters
    aug_params = config.AUGMENTATION_LEVELS[config.get('aug_level', 'light')]
    
    # Handle preprocessing based on backbone
    if config.backbone in ['mobilenet', 'efficientnet']:
        from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
        aug_params = aug_params.copy()
        aug_params['preprocessing_function'] = preprocess_input
    
    # Create generators
    train_gen = ImageDataGenerator(**aug_params)
    val_gen = ImageDataGenerator(**aug_params)
    
    # Determine color mode
    color_mode = config.get('color_mode', 'grayscale')
    
    train_set = train_gen.flow_from_directory(
        os.path.join(data_path, "train"),
        target_size=(config.get('picture_size'), config.get('picture_size')),
        color_mode=color_mode,
        batch_size=config.get('batch_size'),
        class_mode='categorical',
        shuffle=True,
    )
    
    validation_set = val_gen.flow_from_directory(
        os.path.join(data_path, "validation"),
        target_size=(config.get('picture_size'), config.get('picture_size')),
        color_mode=color_mode,
        batch_size=config.get('batch_size'),
        class_mode='categorical',
        shuffle=False,
    )
    
    print("Train image shape:", train_set.image_shape)
    print("Val image shape:  ", validation_set.image_shape)

    print(f"\n✅ Data loaded successfully")
    print (f"Colour Mode: {color_mode}")
    print(f"Training samples: {train_set.n}")
    print(f"Validation samples: {validation_set.n}")
    print(f"Class indices: {train_set.class_indices}")
    
    return train_set, validation_set



In [11]:

# =========================================================================
# Model Building
# =========================================================================

def build_custom_cnn(config):
    """Build custom CNN model"""
    picture_size = config.get('picture_size')
    color_mode = config.get('color_mode')
    channels = 1 if color_mode == 'grayscale' else 3
    
    model = Sequential()
   # Block 1 - Double Conv
    model.add(Conv2D(filters=32, 
                        kernel_size=(3, 3), 
                        padding='same', 
                        input_shape=input_shape,
                        name='conv2d_1'))

    # Block 2 - Double Conv
    model.add(Conv2D(filters=128, 
                      kernel_size=(3, 3), 
                      padding='same',
                      name='conv2d_2'))
    model.add(BatchNormalization(name='bn_2'))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2),name='maxpool2d_2'))
    model.add(Dropout(dropout_rate,name='dropout_2'))
    
    # Block 3 - Double Conv
    model.add(Conv2D(filters=512, 
                      kernel_size=(5, 5), 
                      padding='same',
                      name='conv2d_3'))
    model.add(BatchNormalization(name='bn_3'))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2),name='maxpool2d_3'))
    model.add(Dropout(dropout_rate,name='dropout_3'))
    
    # Block 4 - Double Conv
    model.add(Conv2D(filters=512, 
                      kernel_size=(3, 3), 
                      padding='same',
                      name='conv2d_4'))
    model.add(BatchNormalization(name='bn_4'))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2),name='maxpool2d_4'))
    model.add(Dropout(dropout_rate,name='dropout_4'))

    
    # Block 5 - Double Conv
    model.add(Conv2D(filters=512, 
                      kernel_size=(3, 3), 
                      padding='same',
                      name='conv2d_5'))
    model.add(BatchNormalization(name='bn_5'))
    model.add(Activation('elu'))
    model.add(MaxPooling2D(pool_size=(2, 2),name='maxpool2d_5'))
    model.add(Dropout(dropout_rate,name='dropout_5'))
 
        
    model.add(Flatten())
    
    # Add dense layers
    for units in config.get('dense_units', [512]):
        model.add(Dense(units, activation='relu'))
        model.add(BatchNormalization())
        model.add(Dropout(config.get('dropout_rate')))
    
    model.add(Dense(config.get('no_of_classes'), activation='softmax', dtype='float32'))
    
    return model, None


In [12]:

def build_transfer_model(config, backbone_class):
    """Build transfer learning model"""
    picture_size = config.get('picture_size')
    color_mode = config.get('color_mode')
    channels = 1 if color_mode == 'grayscale' else 3
    
    inputs = Input(shape=(picture_size, picture_size, channels))
    
    # Convert grayscale to RGB if needed
    if channels == 1:
        x = Lambda(lambda z: tf.image.grayscale_to_rgb(z))(inputs)
    else:
        x = inputs
    
    # Load pre-trained backbone
    base_model = backbone_class(
        weights='imagenet',
        include_top=False,
        input_shape=(picture_size, picture_size, 3)
    )
    
    base_model.trainable = False
    x = base_model(x, training=False)
    x = GlobalAveragePooling2D()(x)
    x = Dropout(config.get('dropout_rate'))(x)
    
    # Add dense layers
    for units in config.get('dense_units', [512]):
        x = Dense(units, activation='relu')(x)
        x = Dropout(config.get('dropout_rate'))(x)
    
    outputs = Dense(config.get('no_of_classes'), activation='softmax', dtype='float32')(x)
    
    model = Model(inputs, outputs)
    return model, base_model

def build_model(config):
    """Build model based on configuration"""
    backbone = config.backbone
    
    if backbone == 'custom_cnn':
        return build_custom_cnn(config)
    elif backbone == 'mobilenet':
        return build_transfer_model(config, MobileNetV2)
    elif backbone == 'efficientnet':
        return build_transfer_model(config, EfficientNetB0)
    else:
        raise ValueError(f"Unknown backbone: {backbone}")


In [13]:

# =========================================================================
# Training Utilities
# =========================================================================

def create_callbacks(config):
    """Create training callbacks"""
    callbacks = [
        ModelCheckpoint('best_model.keras', monitor='val_accuracy', save_best_only=True),
        EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
        ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, min_delta=0.0001)
    ]
    
    if config.get('use_lr_schedule'):
        def cosine_annealing(epoch, lr):
            initial_lr = float(config.get('learning_rate'))
            min_lr = 1e-7
            warmup = 5
            total_epochs = config.get('epochs')
            
            if epoch < warmup:
                return initial_lr * (epoch + 1) / warmup
            
            progress = (epoch - warmup) / (total_epochs - warmup)
            lr_out = min_lr + (initial_lr - min_lr) * 0.5 * (1 + math.cos(math.pi * progress))
            return max(lr_out, min_lr)
        
        callbacks.append(LearningRateScheduler(cosine_annealing))
    
    return callbacks

def compute_class_weights(train_set):
    """Compute class weights for imbalanced data"""
    class_weights = compute_class_weight(
        'balanced',
        classes=np.unique(train_set.classes),
        y=train_set.classes
    )
    return dict(enumerate(class_weights))


In [14]:
# =========================================================================
# Run Training 
# =========================================================================

# Simple usage
config = ModelConfig("mobilenet")  # or 'custom_cnn', 'efficientnet'


import socket

if hostname  == "xscape7x": # local machine
    print("Hostname:", hostname)
    print("Overiding settings for xscape7x...")
    # Override any settings
    if config.backbone == "mobilenet":
        config.set("epochs", 1)
        config.set("batch_size", 32)
        config.set("fine_tune_epochs", 1)
        config.set("fine_tune_unfreeze_layers", 1)
    else:
        config.set("epochs", 1)
        config.set("batch_size", 32)
else:                                           # run on remote server
    pass  # No overrides

print("=" * 50)
print(f"Configuration: {config.backbone.upper()}")
print("=" * 50)
for key, value in config.config.items():
    print(f"{key:20}: {value}")
print("=" * 50)


Hostname: xscape7x
Overiding settings for xscape7x...
Configuration: MOBILENET
picture_size        : 96
color_mode          : rgb
batch_size          : 32
epochs              : 1
learning_rate       : 0.0001
dropout_rate        : 0.3
dense_units         : [512, 256]
aug_level           : strong
use_class_weights   : True
use_lr_schedule     : True
fine_tune           : True
fine_tune_epochs    : 1
fine_tune_unfreeze_layers: 1
backbone            : mobilenet
no_of_classes       : 7


In [15]:

# Train
# model, history, train_set, val_set = train_model(config)

# =========================================================================
# Compile Model and Load Data
# =========================================================================

# Setup
data_path = get_data_path()
print(f"Data folder path: {data_path}")

train_set, validation_set = create_data_generators(config, data_path)

print("Train image shape:", train_set.image_shape)
print("Val image shape:  ", validation_set.image_shape)

print(f"\n✅ Data loaded successfully")
print (f"Colour Mode: {config.config.get('color_mode')}")    
print(f"Training samples: {train_set.n}")
print(f"Validation samples: {validation_set.n}")
print(f"Class indices: {train_set.class_indices}")

# Build model
model, base_model = build_model(config)

# Validate shapes
assert train_set.image_shape == model.input_shape[1:], (
    f"Shape mismatch: {train_set.image_shape} vs {model.input_shape[1:]}")

#print ("== Base Model ====")
#base_model.summary()
print ("== Full Model ====")
model.summary()

# Compile
optimizer = Adam(learning_rate=config.get("learning_rate"))
model.compile(
    optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"]
)

print(f"\n{'='*70}")
print(f"Building model with '{config.get('backbone')}' backbone...")
print(f"{'='*70}")

Data folder path: data/images/
Found 28821 images belonging to 7 classes.
Found 7066 images belonging to 7 classes.
Train image shape: (96, 96, 3)
Val image shape:   (96, 96, 3)

✅ Data loaded successfully
Colour Mode: rgb
Training samples: 28821
Validation samples: 7066
Class indices: {'angry': 0, 'disgust': 1, 'fear': 2, 'happy': 3, 'neutral': 4, 'sad': 5, 'surprise': 6}
Train image shape: (96, 96, 3)
Val image shape:   (96, 96, 3)

✅ Data loaded successfully
Colour Mode: rgb
Training samples: 28821
Validation samples: 7066
Class indices: {'angry': 0, 'disgust': 1, 'fear': 2, 'happy': 3, 'neutral': 4, 'sad': 5, 'surprise': 6}
== Full Model ====


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 96, 96, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ mobilenetv2_1.00_96             │ (None, 3, 3, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 1280)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │       655,872 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 7)              │         1,799 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,046,983 (11.62 MB)

 Trainable params: 788,999 (3.01 MB)

 Non-trainable params: 2,257,984 (8.61 MB)


Building model with 'mobilenet' backbone...


In [ ]:
# =========================================================================
# Train the Model Stage 1
# =========================================================================
# Class weights
class_weights = None
if config.get("use_class_weights"):
    class_weights = compute_class_weights(train_set)

# Callbacks
callbacks = create_callbacks(config)
print(f"✅ Callbacks configured: {len(callbacks)} callbacks")

# Training
print(f"Training {config.backbone} model...")
print("\n" + "="*70)
print("🚀 STARTING TRAINING")
print("="*70)
print(f"Target: {config.get('epochs')} epochs with early stopping")
print(f"Batch size: {config.get('batch_size')}")
print(f"Learning rate: {config.get('learning_rate')}")
print(f"Augmentation: {config.get('aug_level')}")
print(f"Backbone: {config.get('backbone')}")
print(f"Class weights: {'Enabled' if class_weights else 'Disabled'}")
print("="*70)

# Stage 1: Train head
history_head = model.fit(
    train_set,
    epochs=config.get("epochs"),
    validation_data=validation_set,
    callbacks=callbacks,
    class_weight=class_weights,
    verbose=1,
)


✅ Callbacks configured: 4 callbacks
Training mobilenet model...

🚀 STARTING TRAINING
Target: 1 epochs with early stopping
Batch size: 32
Learning rate: 0.0001
Augmentation: strong
Backbone: mobilenet
Class weights: Enabled


c:\Users\marty\anaconda3\envs\tf311_env\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


In [ ]:
# =========================================================================
# Train the Model Stage 2 - Fine Tuning
# =========================================================================
# Stage 2: Fine-tune (if transfer learning)
if base_model and config.get("fine_tune"):
    print("Fine-tuning...")

    # Unfreeze layers
    unfreeze_layers = config.get("fine_tune_unfreeze_layers", 30)
    for layer in base_model.layers[unfreeze_layers:]:
        layer.trainable = True

    # Recompile with lower LR
    model.compile(
        optimizer=Adam(learning_rate=config.get("learning_rate") * 0.1),
        loss="categorical_crossentropy",
        metrics=["accuracy"],
    )

    # Continue training
    history = model.fit(
        train_set,
        epochs=config.get("epochs") + config.get("fine_tune_epochs", 0),
        initial_epoch=config.get("epochs"),
        validation_data=validation_set,
        callbacks=callbacks,
        class_weight=class_weights,
        verbose=1,
    )
    



In [ ]:
# =========================================================================
# Save Model
# =========================================================================
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
model_name = f"emotion_recognition_{timestamp}.keras"

if os.path.exists('/kaggle/input')
    output_dir = "/kaggle/working/" + timestamp + "/"  
else:
    output_dir = "outputs/" + timestamp + "/"

os.makedirs(output_dir, exist_ok=True)
out_path = Path(output_dir) / model_name
try:   
    model.save(str(out_path))
    print(f"\n💾 Model saved: {out_path}")
except Exception as e:
    print(f"Error saving model: {e}")

In [ ]:
# =========================================================================
# Evaluate the Model
# =========================================================================
# Evaluate
val_loss, val_acc = model.evaluate(validation_set)
print(f"Validation accuracy: {val_acc:.4f}")

In [ ]:
# =========================================================================
# Extract Values from the Model
# =========================================================================
history = history if history is not None else history_head

# Helper to safely get values
def get_vals(h, key):
    return h.history.get(key, []) if h is not None else []

head_acc = get_vals(history_head, 'accuracy')
head_val_acc = get_vals(history_head, 'val_accuracy')
head_loss = get_vals(history_head, 'loss')
head_val_loss = get_vals(history_head, 'val_loss')

ft_acc = get_vals(history, 'accuracy')
ft_val_acc = get_vals(history, 'val_accuracy')
ft_loss = get_vals(history, 'loss')
ft_val_loss = get_vals(history, 'val_loss')

train_acc = head_acc + ft_acc
val_acc   = head_val_acc + ft_val_acc
train_loss = head_loss + ft_loss
val_loss = head_val_loss + ft_val_loss

In [ ]:

# After training
# history_head, history_ft = train_two_stage(...)

print ("plot both stages combined (recommended so plots show full training curve):")

# Plot
fig, axes = plt.subplots(1, 2, figsize=(15, 5))
axes[0].plot(train_set, label='Train Accuracy', linewidth=2)
axes[0].plot(val_acc, label='Val Accuracy', linewidth=2)
axes[0].set_title('Model Accuracy'); axes[0].legend(); axes[0].grid(True)

axes[1].plot(train_loss, label='Train Loss', linewidth=2)
axes[1].plot(val_loss, label='Val Loss', linewidth=2)
axes[1].set_title('Model Loss'); axes[1].legend(); axes[1].grid(True)

plt.tight_layout()
plt.savefig(os.path.join(output_dir, 'training_curve_head.png'), dpi=300, bbox_inches='tight')
plt.show()
#print(f"📊 Final Validation Accuracy: {val_acc:.4f} ({val_acc*100:.2f}%)")


In [ ]:
# =======================================================================
# Training History Visualization
# =======================================================================

# 1) Get scalar evaluation results from the model (safe)
train_loss, train_acc = model.evaluate(train_set, verbose=0)
val_loss, val_acc = model.evaluate(validation_set, verbose=0)

print("\n" + "="*70)
print("📊 FINAL EVALUATION")
print("="*70)
print(f"Training Accuracy:   {train_acc:.4f} ({train_acc*100:.2f}%)")
print(f"Validation Accuracy: {val_acc:.4f} ({val_acc*100:.2f}%)")
print(f"Training Loss:       {train_loss:.4f}")
print(f"Validation Loss:     {val_loss:.4f}")
print("="*70)

# 2) Consolidate histories (head + fine-tune) for plotting if present
def _get_hist_list(h):
    return h.history if h is not None else {}

head_hist = history_head if 'history_head' in globals() else None
ft_hist = history_ft if 'history_ft' in globals() else None

# Combine metrics safely
def _concat_metric(metric):
    vals = []
    if head_hist is not None:
        vals += head_hist.history.get(metric, [])
    if ft_hist is not None:
        vals += ft_hist.history.get(metric, [])
    return vals

train_acc_list = _concat_metric('accuracy')
val_acc_list = _concat_metric('val_accuracy')
train_loss_list = _concat_metric('loss')
val_loss_list = _concat_metric('val_loss')

def plot_training_history(history_data):
    """Plot training history with better formatting."""
    fig, axes = plt.subplots(1, 2, figsize=(15, 5))
    
    # Accuracy plot
    axes[0].plot(history_data['accuracy'], label='Train Accuracy', linewidth=2)
    axes[0].plot(history_data['val_accuracy'], label='Val Accuracy', linewidth=2)
    axes[0].set_title('Model Accuracy')
    axes[0].set_xlabel('Epoch')
    axes[0].set_ylabel('Accuracy')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)
    
    # Loss plot
    axes[1].plot(history_data['loss'], label='Train Loss', linewidth=2)
    axes[1].plot(history_data['val_loss'], label='Val Loss', linewidth=2)
    axes[1].set_title('Model Loss')
    axes[1].set_xlabel('Epoch')
    axes[1].set_ylabel('Loss')
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, 'training_curve_full.png'), dpi=300, bbox_inches='tight')
    plt.show()


# Prepare history data dictionary for plotting
history_data = {
    'accuracy': train_acc_list,
    'val_accuracy': val_acc_list,
    'loss': train_loss_list,
    'val_loss': val_loss_list
}

# Plot the training history
plot_training_history(history_data)

In [ ]:
# Final evaluation
print("\n" + "="*70)
print("📊 FINAL EVALUATION")
print("="*70)
print(f"Training Accuracy:   {train_acc:.4f} ({train_acc*100:.2f}%)")
print(f"Validation Accuracy: {val_acc:.4f} ({val_acc*100:.2f}%)")
print(f"Training Loss:       {train_loss:.4f}")
print(f"Validation Loss:     {val_loss:.4f}")
print("="*70)

In [ ]:
# =======================================================================
# Enhanced Performance Visualizations
# =======================================================================
# Add this before the visualization section
# before visualization
validation_set.reset()
preds = model.predict(validation_set, verbose=0)
predicted_classes = np.argmax(preds, axis=1)
true_classes = validation_set.classes
cm = confusion_matrix(true_classes, predicted_classes)



# 1. Confusion Matrix
plt.figure(figsize=(6, 5))
sns.heatmap(cm, annot=True, fmt='d', cmap='YlOrRd',
            xticklabels=class_labels,
            yticklabels=class_labels,
            cbar_kws={'label': 'Count'},
            linewidths=0.5,
            linecolor='gray')
plt.title('Confusion Matrix - Emotion Recognition', fontsize=16, fontweight='bold', pad=20)
plt.xlabel('Predicted Emotion', fontsize=13, fontweight='bold')
plt.ylabel('True Emotion', fontsize=13, fontweight='bold')
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()
plt.savefig(os.path.join(out_path, 'confusion_matrix.png'), dpi=300, bbox_inches='tight')
plt.show()

# 2. Normalized Confusion Matrix
cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

plt.figure(figsize=(6, 5))
sns.heatmap(cm_normalized, annot=True, fmt='.2%', cmap='Blues',
            xticklabels=class_labels,
            yticklabels=class_labels,
            cbar_kws={'label': 'Percentage'})
plt.title('Normalized Confusion Matrix (%)', fontsize=16, fontweight='bold', pad=20)
plt.xlabel('Predicted Emotion', fontsize=13, fontweight='bold')
plt.ylabel('True Emotion', fontsize=13, fontweight='bold')
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()
plt.savefig(os.path.join(out_path, 'confusion_matrix_normalized.png'), dpi=300, bbox_inches='tight')
plt.show()

# 3. Classification Report
print("\n" + "="*70)
print("📈 DETAILED CLASSIFICATION METRICS")
print("="*70)
print(classification_report(true_classes, predicted_classes, 
                           target_names=class_labels,
                           digits=4))

# 4. Per-Class Metrics Bar Chart
precision, recall, f1, support = precision_recall_fscore_support(
    true_classes, predicted_classes, labels=range(7)
)

fig, ax = plt.subplots(figsize=(7, 5))
x = np.arange(len(class_labels))
width = 0.25

bars1 = ax.bar(x - width, precision, width, label='Precision', color='#3498db')
bars2 = ax.bar(x, recall, width, label='Recall', color='#2ecc71')
bars3 = ax.bar(x + width, f1, width, label='F1-Score', color='#e74c3c')

ax.set_xlabel('Emotion', fontsize=12, fontweight='bold')
ax.set_ylabel('Score', fontsize=12, fontweight='bold')
ax.set_title('Per-Emotion Performance Metrics', fontsize=14, fontweight='bold')
ax.set_xticks(x)
ax.set_xticklabels(class_labels, rotation=45, ha='right')
ax.legend()
ax.grid(axis='y', alpha=0.3)
ax.set_ylim([0, 1.1])

# Add value labels on bars
for bars in [bars1, bars2, bars3]:
    for bar in bars:
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height,
                f'{height:.2f}',
                ha='center', va='bottom', fontsize=8)

plt.tight_layout()
plt.savefig(os.path.join(out_path, 'per_class_metrics.png'), dpi=300, bbox_inches='tight')
plt.show()

# 5. Summary Table
summary_df = pd.DataFrame({
    'Emotion': class_labels,
    'Precision': [f'{p:.2%}' for p in precision],
    'Recall': [f'{r:.2%}' for r in recall],
    'F1-Score': [f'{f:.2%}' for f in f1],
    'Support': support.astype(int)
})

print("\n" + "="*70)
print("📊 PERFORMANCE SUMMARY TABLE")
print("="*70)
print(summary_df.to_string(index=False))
print("="*70)

# 6. Misclassification Analysis
misclassified = cm.copy()
np.fill_diagonal(misclassified, 0)

top_confusions = []
for i in range(7):
    for j in range(7):
        if i != j:
            top_confusions.append((class_labels[i], class_labels[j], misclassified[i, j]))

top_confusions.sort(key=lambda x: x[2], reverse=True)

print("\n" + "="*70)
print("⚠️  TOP 5 MISCLASSIFICATION PAIRS")
print("="*70)
for true_label, pred_label, count in top_confusions[:5]:
    print(f"{true_label:>10} → {pred_label:<10} : {count:>4} times")
print("="*70)

In [ ]:
# =======================================================================
# Sample Predictions Visualization
# =======================================================================
import random

# Get a random batch from validation set
random_batch_idx = random.randint(0, len(validation_set) - 1)
sample_images, sample_labels = validation_set[random_batch_idx]

# Predict
sample_preds = model.predict(sample_images, verbose=0)
sample_pred_classes = np.argmax(sample_preds, axis=1)
sample_true_classes = np.argmax(sample_labels, axis=1)

# Randomly select 16 indices from the batch
num_samples = min(16, len(sample_images))
random_indices = random.sample(range(len(sample_images)), num_samples)

# Visualize predictions
fig, axes = plt.subplots(4, 4, figsize=(12, 12))
axes = axes.ravel()

for i, idx in enumerate(random_indices):
    axes[i].imshow(sample_images[idx].squeeze(), cmap='gray')
    
    true_label = class_labels[sample_true_classes[idx]]
    pred_label = class_labels[sample_pred_classes[idx]]
    confidence = sample_preds[idx][sample_pred_classes[idx]] * 100

    # Color: green if correct, red if wrong
    color = 'green' if sample_true_classes[idx] == sample_pred_classes[idx] else 'red'
    
    axes[i].set_title(f'True: {true_label}\nPred: {pred_label} ({confidence:.1f}%)', 
                     color=color, fontweight='bold')
    axes[i].axis('off')

plt.suptitle('Sample Predictions on Validation Set', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.savefig(os.path.join(out_path, 'sample_predictions.png'), dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
print("Global policy:", mixed_precision.global_policy())
print("Model dtype policy:", model.dtype_policy)   # or model.input_dtype / model.output_dtype